In [ ]:
### we probably don't need this cell

def tvb_make_sim_file(pse_name, subj_list, param1_name, param1_min, param1_max, param1_num, param2_name, param2_min, param2_max, param2_num):
    import numpy as np
    f = open('sim_info_' + pse_name + '.txt', 'write')
    f.write('SUBJECTS: \t' + subj_list + '\n')
    f.write('Parameter 1: \t' + param1_name + '\n')
    f.write('Parameter 2: \t' + param2_name + '\n')
    f.close()
    p1 = np.linspace(param1_min, param1_max, param1_num)
    p2 = np.linspace(param2_min, param2_max, param2_num)
    np.savetxt('sim_params_' + pse_name + '_param1.txt', p1, fmt='%.4f', delimiter=' ', newline='\n')
    np.savetxt('sim_params_' + pse_name + '_param2.txt', p2, fmt='%.4f', delimiter=' ', newline='\n')
    

In [1]:
# https://groups.google.com/forum/#!topic/tvb-users/ODsL9bkGLHQ

import warnings
warnings.filterwarnings('ignore')
import os, sys, scipy.io, numpy as np
from nipype import Node, Function, Workflow
#from tvb.simulator.lab import *

cwd = os.getcwd()

# https://miykael.github.io/nipype_tutorial/notebooks/basic_workflow.html

# done
def make_pse(parameter_ranges): # done, need to wrap
    import numpy as np
    pse_list = dict(parameter_ranges)
    return pse_list

def make_model(model_name, parameters):# done
    import warnings, pickle, os
    warnings.filterwarnings('ignore')
    from tvb.simulator.lab import models
    import numpy as np
    mod = getattr(models, model_name)
    model_class = mod(**dict(parameters))
    pickle.dump(model_class, open("model_class.p", "wb"))
    model_class = os.path.abspath("model_class.p")
    return model_class

def load_connectivity_mat(in_file, normalize=False):
    import scipy.io, pickle, os
    datamat = scipy.io.loadmat(in_file)
    sc_weights = datamat['sc_weights']
    if normalize:
        sc_weights = sc_weights / sc_weights.max()
    tract_lengths = datamat['tract_lengths']
    scipy.io.savemat('sc_weights.mat',{'sc_weights': sc_weights})
    scipy.io.savemat('tract_lengths.mat',{'tract_lengths': tract_lengths})
    sc_weights = os.path.abspath("sc_weights.mat")
    tract_lengths = os.path.abspath("tract_lengths.mat")
    return sc_weights, tract_lengths
    
def make_connectivity(weights, lengths):
    import warnings, pickle, os, scipy.io
    warnings.filterwarnings('ignore')
    weights_mat = scipy.io.loadmat(weights); weights = weights_mat['sc_weights']
    lengths_mat = scipy.io.loadmat(lengths); lengths = lengths_mat['tract_lengths']
    from tvb.simulator.lab import connectivity
    conn_class = connectivity.Connectivity(weights=weights, tract_lengths=lengths)
    pickle.dump(conn_class, open("conn_class.p", "wb"))
    conn_class = os.path.abspath("conn_class.p")
    return conn_class

def make_integrator(integrator_name, base_dt, noise_type, noise_val):
    import sys, numpy, warnings, pickle, os
    warnings.filterwarnings('ignore')
    sys.modules['mtrand'] = numpy.random.mtrand 
    from tvb.simulator.lab import integrators #, noise
    temp_integrator = getattr(integrators,integrator_name)
    #temp_noise = getattr(noise, noise_type)
    #noise = temp_noise(nsig = np.array(noise_val))
    # integrator_class = temp_integrator(dt = base_dt, noise = noise)
    integrator_class = temp_integrator(dt = base_dt)
    pickle.dump(integrator_class, open("integrator_class.p", "wb"))
    integrator_class = os.path.abspath("integrator_class.p")
    return integrator_class

def make_monitors(monitor_types, periods):
    import warnings, sys, numpy, pickle, os
    warnings.filterwarnings('ignore')
    sys.modules['mtrand'] = numpy.random.mtrand
    from tvb.simulator.lab import monitors
        
    monitor_class = []
    for i in range(len(monitor_types)):
        monitor_tmp = getattr(monitors,monitor_types[i])
        monitor_tmp2 = monitor_tmp(period = periods[i])
        monitor_class.append(monitor_tmp2)
        
    monitor_class = tuple(monitor_class)

    pickle.dump(monitor_class, open("monitor_class.p", "wb"))
    monitor_class = os.path.abspath("monitor_class.p")
    return monitor_class


def run_simulation(out_file, model_input, conn_input, integrator_input, monitor_input, global_coupling = 0.1, conduction_speed=3.0, simulation_length=10000.0):
    import warnings, sys, numpy, pickle, os, scipy.io
    warnings.filterwarnings('ignore')
    sys.modules['mtrand'] = numpy.random.mtrand
    
    model_input = pickle.load(open(model_input, "rb"))
    conn_input = pickle.load(open(conn_input, "rb"))
    integrator_input = pickle.load(open(integrator_input, "rb"))
    monitor_input = pickle.load(open(monitor_input, "rb"))

    from tvb.simulator.lab import *
    wm_coupling = coupling.Linear(a = global_coupling)
    
    # testing things # fix this
    # monitor_input = (monitors.Bold(period=2000.0), monitors.TemporalAverage(period=10.0), monitors.ProgressLogger(period=10000.0))
    # monitor_input = (monitors.Bold(period=2000.0), monitors.TemporalAverage(period=10.0))
    
    sim = simulator.Simulator(model = model_input, connectivity = conn_input, coupling = wm_coupling,
                             integrator = integrator_input, monitors = monitor_input,
                             simulation_length = simulation_length, conduction_speed = conduction_speed)
    
    sim.configure()
    #(bold_time, bold_data), (tavg_time, tavg_data) = sim.run()
    sim_output = sim.run()
    # numpy.save(out_file, data)
    scipy.io.savemat('sim_output.mat',{'sim_output': sim_output})
    abs_out_file = os.path.abspath("sim_output.mat") # fix this
    return abs_out_file

##### NIPYPE PORTION
# https://miykael.github.io/nipype_tutorial/notebooks/basic_function_interface.html

pse_params = Node(
    Function(
        input_names=['parameter_ranges'],
        output_names=['model_class'],
        function=make_pse
    ),
    name='create_pse'
)

model = Node(
    Function(
        input_names=['model_name', 'parameters'],
        output_names=['model_class'],
        function=make_model
    ),
    name='create_model'
)
 
sc_loader = Node(
    Function(
        input_names=['in_file', 'normalize'],
        output_names=['sc_weights', 'tract_lengths'],
        function=load_connectivity_mat
    ),
    name='load_sc_mat'
)

sc = Node(
    Function(
        input_names=['weights', 'lengths'],
        output_names=['conn_class'],
        function=make_connectivity
    ),
    name='create_sc'
)

integrator = Node(
    Function(
        input_names=['integrator_name','base_dt','noise_type','noise_val'],
        output_names=['integrator_class'],
        function=make_integrator
    ),
    name='create_integrator'
)

monitors = Node(
    Function(
        input_names=['monitor_types','periods'],
        output_names=['monitor_class'],
        function=make_monitors
    ),
    name='create_monitors'
)

simulate = Node(
    Function(
        input_names=['out_file', 'model_input', 'conn_input', 'integrator_input', 'monitor_input',
                     'global_coupling', 'conduction_speed', 'simulation_length'],
        output_names=['abs_out_file'],
        function=run_simulation
    ),
    name='create_simulation'
)



In [2]:
# https://miykael.github.io/nipype_tutorial/notebooks/basic_workflow.html
workflow = Workflow(name='tvb_demo', base_dir=os.getcwd())
workflow.connect([
    (model, simulate, [("model_class", "model_input")]),
    (sc_loader, sc, [("sc_weights", "weights"), ("tract_lengths", "lengths")]),
    (sc, simulate, [("conn_class", "conn_input")]),
    (integrator, simulate, [("integrator_class", "integrator_input")]),
    (monitors, simulate, [("monitor_class", "monitor_input")])
])


# NOW DEFINE YOUR INPUTS
# https://miykael.github.io/nipype_tutorial/notebooks/basic_data_input.html
model.inputs.model_name = 'Generic2dOscillator'
model.inputs.parameters = [('a',1), ('b',1)]
# https://miykael.github.io/nipype_tutorial/notebooks/basic_iteration.html
# workflow.model.iterables = ('parameters', [4, 8, 16])
sc_loader.inputs.in_file = cwd + '/input/sub-01_connectivity.mat'
sc_loader.inputs.normalize = False 
integrator.inputs.integrator_name = 'HeunStochastic'
integrator.inputs.base_dt = 0.1
integrator.inputs.noise_type = 'Additive'
integrator.inputs.noise_val = 0.0001
#workflow.integrator.iterables = ('noise', [1, 2, 3, 4])
monitors.inputs.monitor_types = ['Bold', 'TemporalAverage']
monitors.inputs.periods = [2000.0, 10.0]
simulate.inputs.out_file = cwd + '/tvb_test1.mat'
simulate.inputs.global_coupling = 0.1
simulate.inputs.conduction_speed = 2.0
simulate.inputs.simulation_length = 10000.0

# ^ move constants to top node; have initial node with subject list 
# make datasink at the end to clean things up 
#def run_simulation(out_file, model_input, conn_input, integrator_input, monitor_input, global_coupling = 0.1, conduction_speed=2.0, simulation_length=1000.0):


In [3]:
# Write graph of type orig
workflow.write_graph(graph2use='orig', dotfilename='./graph_orig.dot')

# Visualize graph
from IPython.display import Image
Image(filename="graph_orig.png")

IOError: No command "dot" found on host Ludus. Please check that the corresponding package is installed.

In [4]:
#workflow.run('MultiProc', plugin_args={'n_procs': 10})
# import sys, pickle, numpy
# sys.modules['mtrand'] = numpy.random.mtrand
workflow.run()

180809-16:27:05,333 nipype.workflow INFO:
	 Workflow tvb_demo settings: ['check', 'execution', 'logging', 'monitoring']
180809-16:27:05,345 nipype.workflow INFO:
	 Running serially.
180809-16:27:05,347 nipype.workflow INFO:
	 [Node] Setting-up "tvb_demo.load_sc_mat" in "/home/axiezai/neuroha2018/tvb_nipype/tvb_demo/load_sc_mat".
180809-16:27:05,357 nipype.workflow INFO:
	 [Node] Running "load_sc_mat" ("nipype.interfaces.utility.wrappers.Function")
180809-16:27:05,376 nipype.workflow INFO:
	 [Node] Finished "tvb_demo.load_sc_mat".
180809-16:27:05,378 nipype.workflow INFO:
	 [Node] Setting-up "tvb_demo.create_sc" in "/home/axiezai/neuroha2018/tvb_nipype/tvb_demo/create_sc".
180809-16:27:05,391 nipype.workflow INFO:
	 [Node] Running "create_sc" ("nipype.interfaces.utility.wrappers.Function")
180809-16:27:07,743 nipype.workflow INFO:
	 [Node] Finished "tvb_demo.create_sc".
180809-16:27:07,747 nipype.workflow INFO:
	 [Node] Setting-up "tvb_demo.create_model" in "/home/axiezai/neuroha2018/tv

In [ ]:
!nipypecli crash crash-20180809-160454-axiezai-create_simulation-847abe4c-878d-45a5-b9fb-51fc627925f8.pklz

In [ ]:
def make_monitors(monitor_types, periods):
    import warnings, sys, numpy, pickle, os
    warnings.filterwarnings('ignore')
    sys.modules['mtrand'] = numpy.random.mtrand
    from tvb.simulator.lab import monitors
    
    monitor_class = []
    for i in range(len(monitor_types)):
        monitor_tmp = getattr(monitors,monitor_types[i])
        monitor_tmp2 = monitor_tmp(period = periods[i])
        monitor_class.append(monitor_tmp2)
        
    monitor_class = tuple(monitor_class)
    
    return monitor_class

In [ ]:
monitor_class = make_monitors(['Bold', 'TemporalAverage'],[2000.0, 10.0])

In [ ]:
monitor_class

In [ ]:
from tvb.simulator.lab import *
monitor_input = (monitors.Bold(period=2000.0), monitors.TemporalAverage(period=10.0))
monitor_input    

In [ ]:
import pickle
pickle.dump(monitor_class, open("monitor_class.p", "wb"))

In [ ]:
import pickle
monitor_input = pickle.load(open('/mnt/c/Users/easso/docs/neurohackademy/tvb_nipype/tvb_demo/create_monitors/monitor_class.p', "rb"))

In [ ]:
monitor_input

In [ ]:
simulate.outputs


In [ ]:
import scipy.io

In [ ]:
data = scipy.io.loadmat('/home/axiezai/neuroha2018/tvb_nipype/tvb_demo/create_simulation/sim_data.mat')

In [ ]:
data = data['data']


In [ ]:
data[0][1].shape